# Python Client for Cloud Build

Containers are helpful.

> TL;DR
> Use containers to bring together **software** and training **code** so that you can easily launch jobs on different **compute** with different **parameters** to simplify the operations of ML training.

At the moment we train an ML model a lot of things come together to make it happen:
- **compute** in running: CPUs, memory, networking, GPUs on one or more instances
- **software** is running on the compute
    - the required packages are installed with the software
- training **code**/script is launched with the software
- training **data** is read by the training code
- **parameters** that the code uses to configure the training run

It's tempting to develop in an IDE, like JupyterLab here, and then just make the VM behind it much larger.  This note book here is running in JupyterLab hosted on **compute** running **software** and is being used to author **code** that reads **data** using **parameters** set as Python variables.  One of the issues with that is that typing these words just cost `$$$$` and this instance might not be able to run this notebook 10 times in parallel with different **parameters**.  

A better way?  Keep using an enviorment like this to develop our **code** and make sure it works. Just use smaller **compute** and **data** during this development process.  Then, launch a sepearate, managed job, that runs the full training.  How? What if we could instruct a service to take the list of inputs above and run a job and only charge for the compute used for the duration of training? That is exactly what Vertex AI Training is used for.  With that in mind it also helps scale the usefulness of training as a next step:
- specify distributed training, pools of compute instances
- manage hyperparameter tuning with multiple parallel training jobs focusing in on the right values for hyperparameters
- run many training jobs at the same time without managing compute but also controling cost of this scale

Vertex AI has a [list of provided pre-built training containers](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for the most popular frameworks.  They are made available in multiple release versions of the frameworks and with/without [CUDA](https://developer.nvidia.com/cuda-toolkit) already configured and setup for GPU based training.

For Vertex AI Training Custom Jobs you:
- specify the **compute** to use in parameters or as worker pool specs
- provide a URI for a container with the **software** to use
- provide training **code** in one of three ways
    - as a link to a Python script (file.py)
    - as URI to GCS for a Python Source Distribution
    - as a starting point to code already included on the container with the **software**
- provide **data**
    - as a **parameter** specifying the location the **code** can use to retrieve it
    - or build the logic for connecting to the data source into the **code**

If we learn the skill of building a derivative containers that packages our desired **software** and install additional packages while also holding a copy of our **code** and maybe even our **parameters** then this ML training jobs become very simple to incorporate in our workflow!

That is this notebooks goal.

---
**We will use [Cloud Build](https://cloud.google.com/build) to construct containers.**

- [API Overview](https://cloud.google.com/build/docs/api)
    - REST API, gcloud CLI, and Client Libraries for Go, Java, Node.js, and Python
- [Python Client for Cloud Build API](https://github.com/googleapis/python-cloudbuild)
- [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/cloudbuild/latest)

---
**We will store built containers in [Artifact Registry](https://cloud.google.com/artifact-registry).**

- [API Overview](https://cloud.google.com/artifact-registry/docs/apis)
- [Python Client for Artifact Registry API](https://github.com/googleapis/python-artifact-registry)
- [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/artifactregistry/latest)

---
**Notes on Python and Google Cloud:**

Google Cloud APIs can be used with the [Google Cloud Python Client](https://github.com/googleapis/google-cloud-python).  The client has [libraries](https://github.com/googleapis/google-cloud-python#libraries) for Google Cloud services.  The documentation for each library is centralized in the [Python Cloud Client Libraries](https://cloud.google.com/python/docs/reference) reference documentation.
- Also helpful: [Getting started with Python](https://cloud.google.com/python/docs/getting-started) in Google Cloud

---
## Package Installs (if needed)

This notebook uses the Python Clients for
- Google Service Usage
- Artifact Registry
- Cloud Build

The cells below check to see if the required Python libraries are installed.  If any are not it will print a message to do the install with the associated pip command to use.  These install must be completed before continuing this notebook.

In [1]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')

In [2]:
try:
    import google.cloud.artifactregistry_v1
except ImportError:
    print('You need to pip install google-cloud-artifact-registry')

In [3]:
try:
    import google.cloud.devtools.cloudbuild
except ImportError:
    print('You need to pip install google-cloud-build')

---
## Setup

inputs:

In [4]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [5]:
REGION = 'us-central1'
EXPERIMENT = 'build'

packages:

In [11]:
import os, shutil

from google.cloud import service_usage_v1
from google.cloud.devtools import cloudbuild
from google.cloud import artifactregistry_v1

clients:

In [12]:
service_usage = service_usage_v1.ServiceUsageClient()



parameters:

In [13]:
DIR = f'temp/{EXPERIMENT}'

environment:
- remote directory named DIR if exists
- create directory DIR
- check for existance of DIR
- list contents of directory one level higher than DIR

In [14]:
shutil.rmtree(DIR, ignore_errors = True)
os.mkdir(DIR)
print(os.path.exists(DIR))
os.listdir(DIR + '/../')

True


['job-parms', 'multi', 'gcs', 'build']

---
## Enable APIs

Using Cloud Build and Artifact Registry requires enabling these APIs for the Google Cloud Project.

Options for enabeling these.  In this notebook (2) is used.
 1. Use the APIs & Services page in the console: https://console.cloud.google.com/apis
     - `+ Enable APIs and Services`
     - Search for Cloud Build and Enable
     - Search for Artifact Registry and Enable
 2. Use [Google Service Usage](https://cloud.google.com/service-usage/docs) API from Python
     - [Python Client For Service Usage](https://github.com/googleapis/python-service-usage)
     - [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/serviceusage/latest)
     
The following code cells use the Service Usage Client to:
- get the state of the service
- if 'DISABLED':
    - Try enabling the service and return the state after trying
- if 'ENABLED' print the state for confirmation

### Artifact Registry

In [15]:
artifactregistry = service_usage.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
    )
).state.name


if artifactregistry == 'DISABLED':
    print(f'Artifact Registry is currently {artifactregistry} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = service_usage.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/artifactregistry.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Artifact Registry is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Artifact Registry already enabled for project: {PROJECT_ID}')

Artifact Registry already enabled for project: statmike-mlops-349915


### Cloud Build

In [16]:
cloudbuild = service_usage.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/cloudbuild.googleapis.com'
    )
).state.name


if cloudbuild == 'DISABLED':
    print(f'Cloud Build is currently {cloudbuild} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = service_usage.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/cloudbuild.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'Cloud Build is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'Cloud Build already enabled for project: {PROJECT_ID}')

Cloud Build already enabled for project: statmike-mlops-349915


---
---
enable cloud build
    list?
enable cloud artifact registry
    list, create registry

build container with pip installs
run with script on aiplatform

build container with pip and copy of script
run on aiplatform

build source distro from code
build containerr with source distro
run on aiplatform

build container from GitHub repo + folder + commitID
run on aiplatform
---
---